# ♥Lets Build food recognition and calorie estimation model using  ResNet model built from scratch !

# import Libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and Preprocess the Food-101 Dataset:

In [ ]:

data_dir = '/kaggle/input/food-101/food-101.zip'

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


# Define the Residual Block:

In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x


# Lets Build ResNet Model From Scratch ! :

In [ ]:
def build_resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Add residual blocks
    for filters in [64, 128, 256, 512]:
        x = residual_block(x, filters)
        x = residual_block(x, filters)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

input_shape = (224, 224, 3)
num_classes = 101  # Food-101 dataset has 101 classes
model = build_resnet(input_shape, num_classes)


# Compile and Train the Model:

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


# Estimate Calorie Content: 
To estimate calorie content, you can add a regression head to the model. Here's an example of how to modify the model to include both classification and regression outputs:

In [ ]:
def build_resnet_with_calorie_estimation(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Add residual blocks
    for filters in [64, 128, 256, 512]:
        x = residual_block(x, filters)
        x = residual_block(x, filters)

    x = layers.GlobalAveragePooling2D()(x)
    classification_output = layers.Dense(num_classes, activation='softmax', name='classification_output')(x)
    calorie_output = layers.Dense(1, activation='linear', name='calorie_output')(x)

    model = models.Model(inputs, [classification_output, calorie_output])
    return model

model = build_resnet_with_calorie_estimation(input_shape, num_classes)

model.compile(
    optimizer='adam',
    loss={'classification_output': 'categorical_crossentropy', 'calorie_output': 'mean_squared_error'},
    metrics={'classification_output': 'accuracy', 'calorie_output': 'mse'}
)

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
